# Анализ базы данных на SQL

В нашем распоряжении база данных сервиса для чтения книг. В базе информация о книгах, авторах, издательстве, пользовательских оценках книг и пользовательских обзорах. 

Необходимо:

   -  Посчитать, сколько книг вышло после 1 января 2000 года;
   -  Для каждой книги посчитать количество обзоров и среднюю оценку;
   -  Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
   -  Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
   -  Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
   
Подключимся к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': '1111', # имя пользователя
'pwd': '1111', # пароль
'host': '1111.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Напишем функцию, выводящую таблицы и выведем их.

In [2]:
def show_table(table_name, con):
    query = '''SELECT * FROM ''' + table_name + ''' LIMIT 5'''
    table = pd.io.sql.read_sql(sql=text(query), con = con)
    display(table)

In [3]:
show_table('''books''', con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
show_table('''authors''', con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
show_table('''publishers''', con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
show_table('''ratings''', con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
table = show_table('''reviews''', con)
table

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Подсчет количества книг, вышедших после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books 
WHERE publication_date > '2000-01-01'
LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


**После 1 января 2000 года вышло 819 книг.**

#### Подсчет количества обзоров и средней оценки для каждой книги


In [9]:
query = '''

SELECT b.book_id, b.title, COUNT(DISTINCT r.review_id), AVG(rat.rating)
FROM books b JOIN reviews r ON b.book_id = r.book_id JOIN ratings rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY b.book_id

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,avg
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
989,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
990,997,Xenocide (Ender's Saga #3),3,3.400000
991,998,Year of Wonders,4,3.200000
992,999,You Suck (A Love Story #2),2,4.500000


**Получили таблицу с количеством обзоров и средней оценкой пользователей.**

#### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц


In [10]:
query = '''
SELECT p.publisher, COUNT(b.book_id)
FROM books b JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


**Наибольшее количество книг более 50 страниц выпустило издательство Penguin Books**.

#### Определение автора с самой высокой средней оценкой книг


In [11]:
query = '''
WITH q AS (
            SELECT b.book_id, b.author_id, COUNT(rating)
            FROM authors a JOIN books b ON a.author_id = b.author_id JOIN ratings r ON b.book_id = r.book_id
            GROUP BY b.book_id
            HAVING COUNT(rating) >= 50)

SELECT a.author_id, a.author, AVG(r.rating)
FROM authors a JOIN q ON a.author_id = q.author_id JOIN ratings r ON q.book_id = r.book_id
GROUP BY a.author_id
ORDER BY avg DESC
LIMIT 1

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg
0,236,J.K. Rowling/Mary GrandPré,4.287097


**Автор с самой высокой средней оценкой книг c 50 и более оценками - J.K. Rowling/Mary GrandPré.**

#### Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''
WITH q AS (
            SELECT username, COUNT(rating)
            FROM ratings 
            GROUP BY username
            HAVING COUNT(rating) > 48)
SELECT AVG(count)
FROM (SELECT q.username, COUNT(review_id)
FROM reviews r JOIN q ON r.username = q.username
GROUP BY q.username) AS q2

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


**Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24 отзыва.**